In [11]:
!pip install pyro-ppl

import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np
import matplotlib.pyplot as plt

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer import MCMC, NUTS, Predictive

In [15]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

device: cuda:0


In [16]:
pyro.set_rng_seed(42)

class BayesianNN(PyroModule):
    def __init__(self):
        super().__init__()
        self.layer1 = PyroModule[nn.Linear](1,5)
        self.layer1.weight = PyroSample(dist.Normal(0.,1.).expand([5,1]).to_event(2))
        self.layer1.bias = PyroSample(dist.Normal(0.,1.).expand([5]).to_event(1))

        self.layer2 = PyroModule[nn.Linear](5,5)
        self.layer2.weight = PyroSample(dist.Normal(0.,1.).expand([5,5]).to_event(2))
        self.layer2.bias = PyroSample(dist.Normal(0.,1.).expand([5]).to_event(1))

        self.layer3 = PyroModule[nn.Linear](5,1)
        self.layer3.weight = PyroSample(dist.Normal(0.,1.).expand([1,5]).to_event(2))
        self.layer3.bias = PyroSample(dist.Normal(0.,1.).expand([1]).to_event(1))

        self.activation = nn.Tanh()

    def forward(self,t):
        t = t.view(-1,1)

        h1 = self.activation(self.layer1(t))
        h2 = self.activation(self.layer2(h1))

        u_pred = self.layer3(h2).squeeze(-1)

        return u_pred

     


In [18]:
# Generate Collocation Points

N_f = 1000
t_f = torch.linspace(0,5,N_f).view(-1,1).to(device).requires_grad_(True)

# Generate Initial Points

t_ic = torch.tensor([0.0], requires_grad=True).to(device)

In [19]:
# Physics Likelihood

def physics(t_f,t_ic,lambda_phy = 100.0, lambda_ic = 100.0):
    model = BayesianNN()
    u = model(t_f)
    du_dt = torch.autograd.grad(u,t_f,grad_outputs=torch.ones_like(u),create_graph=True)[0]

    res_pde = du_dt - t_f

    loss_pde = torch.mean(res_pde**2)

    pyro.factor('physics loss', -lambda_phy * loss_pde)


    u_ic = model(t_ic)

    res_ic = u_ic - 0.0

    loss_ic = torch.mean(res_ic ** 2)

    pyro.factor('initial condition loss', -lambda_ic * loss_ic)


    

In [22]:

nuts_kernel = NUTS(physics)
mcmc = MCMC(nuts_kernel, num_samples=100, warmup_steps=50)
mcmc.run(t_f, t_ic)


samples = mcmc.get_samples()

Warmup:   0%|          | 0/150 [00:00, ?it/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)
     Trace Shapes:      
      Param Sites:      
     Sample Sites:      
layer1.weight dist | 5 1
             value | 5 1
  layer1.bias dist | 5  
             value | 5  